In [2]:
#!/usr/bin/env python3
"""
Option 1: Feature Enhancements – MapQuest Directions App
=======================================================

This script enhances the standard MapQuest Directions lab by adding:
  • Pretty, colored console output (falls back to plain text if Rich is not installed)
  • Choice of units: metric or imperial (--units)
  • Route types: fastest, shortest, pedestrian, bicycle (--route-type)
  • Avoids: tolls, highways, ferries, unpaved, seasonal, countryCrossing (--avoid)
  • Step-by-step table with distance, time, maneuver, street name
  • Summary (total distance, duration, fuel estimate)
  • Alternative output formats: table (default) or JSON (--format)
  • Save results to a JSON file (--out)
  • Simple on-disk caching to speed up repeated queries (--cache ./.mq_cache)

Dependencies:
  - requests (pip install requests)
  - rich (optional, for colored output; pip install rich)
  - tabulate (optional, for better tables; pip install tabulate)

API Key:
  - Set environment variable MAPQUEST_KEY or pass --key <YOUR_KEY>

Examples:
  python mapquest_enhanced.py --from "Sydney NSW" --to "Melbourne VIC" --units metric --route-type fastest --avoid tolls,highways
  python mapquest_enhanced.py --from "Parramatta" --to "Bondi Beach" --format json --out route.json
  python mapquest_enhanced.py --from "UNSW Sydney" --to "Sydney Opera House" --cache ./.mq_cache

Author: Carl, Maria, Aayush
"""
from __future__ import annotations
import argparse
import hashlib
import json
import os
import pathlib
import sys
import time
from typing import Any, Dict, List, Optional, Tuple

import requests

# Optional deps
try:
    from rich.console import Console
    from rich.table import Table
    from rich.panel import Panel
    from rich import box
    RICH = True
    console = Console()
except Exception:
    RICH = False
    console = None

try:
    from tabulate import tabulate
    TABULATE = True
except Exception:
    TABULATE = False

BASE_URL = "https://www.mapquestapi.com/directions/v2/route"

# ------------------------------
# Utility & formatting helpers
# ------------------------------

def km_to_miles(km: float) -> float:
    return km * 0.621371

def miles_to_km(mi: float) -> float:
    return mi / 0.621371

def seconds_to_hms(seconds: int) -> Tuple[int, int, int]:
    h = seconds // 3600
    m = (seconds % 3600) // 60
    s = seconds % 60
    return h, m, s


def fmt_duration(seconds: int) -> str:
    h, m, s = seconds_to_hms(seconds)
    if h:
        return f"{h}h {m}m {s}s"
    if m:
        return f"{m}m {s}s"
    return f"{s}s"


def fmt_distance(value_mi: float, units: str) -> str:
    if units == "metric":
        return f"{miles_to_km(value_mi):.2f} km"
    return f"{value_mi:.2f} mi"


# ------------------------------
# Simple file cache
# ------------------------------

def cache_key(payload: Dict[str, Any]) -> str:
    blob = json.dumps(payload, sort_keys=True).encode()
    return hashlib.sha256(blob).hexdigest()


def cache_load(cache_dir: Optional[str], key: str) -> Optional[Dict[str, Any]]:
    if not cache_dir:
        return None
    p = pathlib.Path(cache_dir) / f"{key}.json"
    if p.exists():
        try:
            return json.loads(p.read_text())
        except Exception:
            return None
    return None


def cache_save(cache_dir: Optional[str], key: str, data: Dict[str, Any]) -> None:
    if not cache_dir:
        return
    p = pathlib.Path(cache_dir)
    p.mkdir(parents=True, exist_ok=True)
    f = p / f"{key}.json"
    f.write_text(json.dumps(data, indent=2))


# ------------------------------
# Core: call MapQuest Directions API
# ------------------------------

def call_mapquest(api_key: str, frm: str, to: str, units: str, route_type: str, avoid: List[str]) -> Dict[str, Any]:
    # MapQuest expects unit in [k] (kilometers) or [m] (miles)
    unit = "k" if units == "metric" else "m"

    # Avoids mapping
    avoids = {
        "tolls": "Toll road",
        "highways": "Limited Access",
        "ferries": "Ferry",
        "unpaved": "Unpaved",
        "seasonal": "Approximate Seasonal Closure",
        "countryCrossing": "Country Border Crossing",
    }

    avoid_links = [avoids[a] for a in avoid if a in avoids]

    payload = {
        "key": api_key,
        "from": frm,
        "to": to,
        "unit": unit,
        "routeType": route_type,  # fastest, shortest, pedestrian, bicycle
        "narrativeType": "micro",
        "ambiguities": "ignore",
        "doReverseGeocode": True,
        "fullShape": False,
        "generalize": 0,
    }

    if avoid_links:
        payload["avoids"] = ",".join(avoid_links)

    return _http_get(BASE_URL, payload)


def _http_get(url: str, params: Dict[str, Any]) -> Dict[str, Any]:
    r = requests.get(url, params=params, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"MapQuest API error: HTTP {r.status_code} -> {r.text[:200]}")
    data = r.json()
    if data.get("info", {}).get("statuscode", 0) != 0:
        raise RuntimeError(f"MapQuest API returned status {data.get('info', {}).get('statuscode')}: {data.get('info', {}).get('messages')}")
    return data


# ------------------------------
# Rendering
# ------------------------------

def render_summary(data: Dict[str, Any], units: str) -> Dict[str, Any]:
    route = data["route"]
    distance_mi = float(route.get("distance", 0.0))
    time_sec = int(route.get("time", 0))
    fuel_used_gal = float(route.get("fuelUsed", 0.0))  # may be 0 for pedestrian/bicycle

    summary = {
        "from": route.get("locations", [{}])[0].get("adminArea5", "Unknown"),
        "to": route.get("locations", [{}])[-1].get("adminArea5", "Unknown"),
        "distance": fmt_distance(distance_mi, units),
        "duration": fmt_duration(time_sec),
        "fuel": f"{fuel_used_gal:.2f} gal" if units == "imperial" else f"{fuel_used_gal * 3.78541:.2f} L",
        "has_toll": bool(route.get("hasTollRoad", False)),
        "has_highways": bool(route.get("hasHighway", False)),
        "route_type": route.get("routeType"),
    }
    return summary


def render_maneuvers(data: Dict[str, Any], units: str) -> List[Dict[str, str]]:
    legs = data.get("route", {}).get("legs", [])
    rows: List[Dict[str, str]] = []
    for leg in legs:
        for m in leg.get("maneuvers", []):
            mi = float(m.get("distance", 0.0))
            rows.append({
                "Step": str(m.get("index", "")),
                "Maneuver": m.get("narrative", ""),
                "Street": m.get("streets", [""])[0] if m.get("streets") else "",
                "Distance": fmt_distance(mi, units),
                "Time": fmt_duration(int(m.get("time", 0))),
            })
    return rows


def print_summary(summary: Dict[str, Any]) -> None:
    if RICH:
        panel = Panel(
            f"From: [bold]{summary['from']}[/bold]\n"
            f"To:   [bold]{summary['to']}[/bold]\n"
            f"Distance: [cyan]{summary['distance']}[/cyan]\n"
            f"Duration: [green]{summary['duration']}[/green]\n"
            f"Fuel:     {summary['fuel']}\n"
            f"Route:    {summary['route_type']}\n"
            f"Highways: {'Yes' if summary['has_highways'] else 'No'} | Tolls: {'Yes' if summary['has_toll'] else 'No'}",
            title="Route Summary",
            box=box.ROUNDED,
        )
        console.print(panel)
    else:
        print("== Route Summary ==")
        for k in ("from", "to", "distance", "duration", "fuel", "route_type", "has_highways", "has_toll"):
            print(f"{k}: {summary[k]}")


def print_maneuvers_table(rows: List[Dict[str, str]]) -> None:
    if not rows:
        print("No maneuvers returned.")
        return

    if RICH:
        table = Table(title="Turn‑by‑Turn Directions", box=box.SIMPLE_HEAVY)
        for col in ("Step", "Maneuver", "Street", "Distance", "Time"):
            table.add_column(col, overflow="fold")
        for r in rows:
            table.add_row(r["Step"], r["Maneuver"], r["Street"], r["Distance"], r["Time"])
        console.print(table)
    elif TABULATE:
        headers = ["Step", "Maneuver", "Street", "Distance", "Time"]
        print(tabulate([[r[h] for h in headers] for r in rows], headers=headers, tablefmt="github"))
    else:
        # Plain fallback
        print("Step | Maneuver | Street | Distance | Time")
        for r in rows:
            print(f"{r['Step']} | {r['Maneuver']} | {r['Street']} | {r['Distance']} | {r['Time']}")


# ------------------------------
# CLI
# ------------------------------

def parse_args() -> argparse.Namespace:
    p = argparse.ArgumentParser(description="Enhanced MapQuest Directions CLI")
    p.add_argument("--from", dest="frm", required=True, help="Origin address or place name")
    p.add_argument("--to", dest="to", required=True, help="Destination address or place name")
    p.add_argument("--key", default=os.getenv("MAPQUEST_KEY", ""), help="MapQuest API key (or set MAPQUEST_KEY)")

    p.add_argument("--units", choices=["metric", "imperial"], default="metric", help="Distance units")
    p.add_argument("--route-type", choices=["fastest", "shortest", "pedestrian", "bicycle"], default="fastest")
    p.add_argument("--avoid", default="", help="Comma-separated avoids: tolls,highways,ferries,unpaved,seasonal,countryCrossing")

    p.add_argument("--format", choices=["table", "json"], default="table", help="Output format")
    p.add_argument("--out", default="", help="Write full API response to JSON file")

    p.add_argument("--cache", default="", help="Cache directory path (e.g., ./.mq_cache)")
    return p.parse_args()


# ------------------------------
# Main
# ------------------------------

def main() -> None:
    args = parse_args()

    if not args.key:
        print("ERROR: Provide a MapQuest API key via --key or MAPQUEST_KEY env var.")
        sys.exit(2)

    avoid_list = [a.strip() for a in args.avoid.split(",") if a.strip()]

    # Build a cache key around all query params that affect the response
    query_payload = {
        "from": args.frm,
        "to": args.to,
        "units": args.units,
        "routeType": args.route_type,
        "avoid": ",".join(sorted(avoid_list)),
    }

    key = cache_key(query_payload)
    data = cache_load(args.cache, key)

    if data is None:
        try:
            data = call_mapquest(args.key, args.frm, args.to, args.units, args.route_type, avoid_list)
            cache_save(args.cache, key, data)
        except Exception as e:
            print(f"ERROR: {e}")
            sys.exit(3)

    # Optionally write full payload for reference/reproducibility
    if args.out:
        try:
            pathlib.Path(args.out).write_text(json.dumps(data, indent=2))
            if RICH:
                console.print(f"[green]Saved full API response to[/green] {args.out}")
            else:
                print(f"Saved full API response to {args.out}")
        except Exception as e:
            print(f"Warning: could not write {args.out}: {e}")

    # Render
    summary = render_summary(data, args.units)
    maneuvers = render_maneuvers(data, args.units)

    if args.format == "json":
        print(json.dumps({"summary": summary, "maneuvers": maneuvers}, indent=2))
        return

    print_summary(summary)
    print_maneuvers_table(maneuvers)

    # Helpful tips for the report/presentation
    if RICH:
        tips = (
            "Tips: Use --format json for programmatic consumption; "
            "--avoid tolls,highways to compare routes; set MAPQUEST_KEY in your shell profile."
        )
        console.print(Panel(tips, title="Notes", box=box.SQUARE))
    else:
        print("Notes: Use --format json for programmatic consumption; --avoid tolls,highways to compare routes;")


if __name__ == "__main__":
    main()


usage: colab_kernel_launcher.py [-h] --from FRM --to TO [--key KEY]
                                [--units {metric,imperial}]
                                [--route-type {fastest,shortest,pedestrian,bicycle}]
                                [--avoid AVOID] [--format {table,json}]
                                [--out OUT] [--cache CACHE]
colab_kernel_launcher.py: error: the following arguments are required: --from, --to
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/lib/python3.12/argparse.py", line 1943, in _parse_known_args2
    namespace, args = self._parse_known_args(args, namespace, intermixed)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/argparse.py", line 2230, in _parse_known_args
    raise ArgumentError(None, _('the following arguments are required: %s') %
argparse.ArgumentError: the following arguments are required: --from, --to

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-3665024524.py", line 344, in <cell line: 0>
    main()
  File "/tmp/ipython-input-3665024524.py", line 282, in main
    args = parse_args()
           ^^^^^^^^^^^^
  File "/tmp/ipython-input-3665024524.py", line 274, in par

TypeError: object of type 'NoneType' has no len()